In [ ]:
import numpy as np
from ml.algorithms.optimization import GradientDescent, StochasticGradientDescent
from ml.algorithms.normalization import MinMaxScaler
from ml.models.linear import PolynomialRegression, LinearRegression

In [ ]:
linear_weights_expected = np.array([[2.0, 3.0]])

x = np.array([
    [3.0], 
    [6.0], 
    [7.0]
])
y = np.array([
    [11.0], 
    [20.0], 
    [23.0]
])

epochs = 5000
learning_rate = 0.05
regularization = 0

In [ ]:
def polyreg(x, y, solver, epochs, learning_rate, regularization):
    regressor = PolynomialRegression(2)
    regressor.fit(x, y, solver, epochs, learning_rate, regularization, scaler=MinMaxScaler)

    print("Learned parameters: ", regressor.weights(0), "Expected: ", "??")


gd = GradientDescent()
polyreg(x, y, gd, epochs, learning_rate, regularization)

sgd = StochasticGradientDescent()
polyreg(x, y, sgd, epochs, learning_rate, regularization)

In [ ]:
def linreg(x, y, solver, epochs, learning_rate, regularization):
    regressor = LinearRegression()
    regressor.fit(x, y, solver, epochs, learning_rate, regularization)

    print("Learned parameters: ", regressor.weights(0), "Expected: ", linear_weights_expected)

gd = GradientDescent()
linreg(x, y, gd, epochs, learning_rate, regularization)

In [ ]:
import numpy as np
from ml.dataset import load_csv, split_train_test

np.random.seed(3)

X, y = load_csv('./datasets/california.csv')
X_train, y_train, X_test, y_test = split_train_test(X, y, 0.8, shuffle=True)

In [ ]:
import numpy as np

from ml.model_selection import GridSearchCV
from ml.models.linear import PolynomialRegression

from ml.algorithms.normalization import MinMaxScaler
from ml.algorithms.optimization import GradientDescent, OrdinaryLeastSquares

grid = GridSearchCV()

for p in [3, 2, 1, 4]:
    for lr in [0.2, 0.3, 0.1]:
        grid.add(
            PolynomialRegression, {
                "solver": GradientDescent(epochs=4000, learning_rate=0.1, regularization=0.0), 
                "degree": p,
                "dataScaler": MinMaxScaler
            }
        )

    # grid.add(
    #     PolynomialRegression, {
    #         "solver": OrdinaryLeastSquares(regularization=0.0), 
    #         "degree": p,
    #         "dataScaler": MinMaxScaler
    #     }
    # )

best = grid.search(X_train, y_train, num_folds=4, shuffle=True)
print("Best model:", best)
print("Learning rate:", best[1]['solver'].learning_rate)

In [ ]:
from ml.stats import rmse

model = best[0]
model_args = best[1]
poly = model(**model_args)

poly.fit(X_train, y_train)
print(rmse(y_test, poly.predict(X_test)))

In [ ]:
for i in range(len(grid.candidates)):
    model = grid.candidates[i][0]
    model_args = grid.candidates[i][1]
    poly = model(**model_args)

    poly.fit(X_train, y_train)
    print(rmse(y_test, poly.predict(X_test)))